In [5]:
from azure.cosmos import CosmosClient, exceptions, PartitionKey
from azure.cosmos.exceptions import CosmosResourceNotFoundError, CosmosHttpResponseError

import json
from dotenv import load_dotenv
import os

# carrega variaveis de ambiente

In [6]:
load_dotenv()
URL = os.getenv('nosql-cosmos-uri')
KEY = os.getenv('nosql-cosmos-pk')
cliente = CosmosClient(URL, credential=KEY)

In [7]:
DATABASE_NAME = "DB-1"
database = cliente.get_database_client(DATABASE_NAME)
CONTAINER_NAME = 'container-1'
container = database.get_container_client(CONTAINER_NAME)

In [4]:
container

<ContainerProxy [dbs/DB-docs-json/colls/container-docs-json-05072024]>

In [ ]:
from azure.cosmos import CosmosClient
import os
from dotenv import load_dotenv

# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

# Obter as variáveis de ambiente
cosmos_db_account_endpoint = os.getenv('nosql-cosmos-uri')
cosmos_db_account_key = os.getenv('nosql-cosmos-pk')

# Criar cliente do Cosmos DB
client = CosmosClient(cosmos_db_account_endpoint, cosmos_db_account_key)

# Listar bancos de dados
databases = list(client.list_databases())
print("Databases:", databases)

# Verificar se o banco de dados específico existe
database_name = "DB-docs-json-05072024"
database_exists = any(db['id'] == database_name for db in databases)
if not database_exists:
    print(f"Database {database_name} não encontrado.")
else:
    print(f"Database {database_name} encontrado.")
    
    # Listar containers no banco de dados
    database = client.get_database_client(database_name)
    containers = list(database.list_containers())
    print("Containers:", containers)

    container_name = 'container-docs-json-05072024'
    container_exists = any(container['id'] == container_name for container in containers)
    if not container_exists:
        print(f"Container {container_name} não encontrado.")
    else:
        print(f"Container {container_name} encontrado.")


# OPERAÇÕES CRUD DATABASES

### cria database

In [9]:
try:
    database = cliente.create_database_if_not_exists(input("Digite o nome que deseja dar ao seu DataBase: "))
except exceptions.CosmosResourceExistsError:
    database = cliente.get_database_client(input("Digite o nome que deseja dar ao seu DataBase: "))

### lista databases

In [9]:
databases = cliente.list_databases()

# imprimindo databases
for db in databases:
    print(f"Database ID: {db['id']}")
    DATABASE_NAME = db['id']

Database ID: DB-docs-json-05072024


### lista database (container)

In [ ]:
databases = cliente.list_databases()

# Print the databases
for db in databases:
    print(f"{db['id']}/")
    DATABASE_NAME = db['id']


try:
    # Acessa o banco de dados
    database = cliente.get_database_client(DATABASE_NAME)
    
    # Lista os containers no banco de dados
    containers = list(database.list_containers())
    
    # Imprime os nomes dos containers
    for container in containers:
        print(container['id'])
except exceptions.CosmosResourceNotFoundError:
    print(f"O banco de dados {DATABASE_NAME} não foi encontrado.")
except Exception as e:
    print(f"Erro ao listar containers: {str(e)}")


### exclui database

In [30]:
database_id = input("Digite o Nome do DataBase que deseja excluir: ")

# Delete the database
try:
    cliente.delete_database(database_id)
    print(f"Database '{database_id}' deleted successfully.")
except CosmosResourceNotFoundError:
    print(f"Error: Database '{database_id}' not found.")
except CosmosHttpResponseError as e:
    print(f"HTTP error occurred: {e.status_code} - {e.message}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Error: Database 'seila' not found.


# OPERAÇÕES CRUD CONTAINERS

### Cria container

In [14]:
try:
    container = database.create_container_if_not_exists(
        id=input("Digite o nome que deseja dar ao seu container: "), 
        partition_key=PartitionKey(path="/documento", kind='Hash')
    )
except exceptions.CosmosResourceExistsError:
    container = database.get_container_client(CONTAINER_NAME)
except exceptions.CosmosHttpResponseError:
    raise

### lista containers

In [ ]:
try:
    # Acessa o banco de dados
    database = cliente.get_database_client(DATABASE_NAME)
    
    # Lista os containers no banco de dados
    containers = list(database.list_containers())
    
    # Imprime os nomes dos containers
    for container in containers:
        print(container['id'])
except exceptions.CosmosResourceNotFoundError:
    print(f"O banco de dados {DATABASE_NAME} não foi encontrado.")
except Exception as e:
    print(f"Erro ao listar containers: {str(e)}")


### exclui containers

In [17]:

try:
    # Acessa o banco de dados
    database = cliente.get_database_client(DATABASE_NAME)
    
    # Solicita o nome do container a ser excluído
    container_id = input("Digite o nome do container que deseja excluir: ")

    # Verifica se o container existe
    try:
        container = database.get_container_client(container_id)
        # Tenta excluir o container
        database.delete_container(container_id)
        print(f"Container '{container_id}' excluído com sucesso.")
    except exceptions.CosmosResourceNotFoundError:
        print(f"Container '{container_id}' não encontrado.")
    except Exception as e:
        print(f"Erro ao excluir container '{container_id}': {str(e)}")

except exceptions.CosmosResourceNotFoundError:
    print(f"O banco de dados {DATABASE_NAME} não foi encontrado.")
except Exception as e:
    print(f"Erro ao acessar o banco de dados: {str(e)}")

Container 'seila' excluído com sucesso.


# OPERAÇÕES CRUD DOCUMENTO

#### AZURE OPEN AI SERVICE    -  deploy-teste-acordao

In [7]:
from get_embeddings import get_embeddings

In [16]:
text = 'sei la, apenas uma frase qualquer pra textar a função get_embeddings'
embedding, modelUsed = get_embeddings(text)
print (modelUsed)
for i in range(5):
    print(embedding[i])

text-embedding-3-large
-0.010189319960772991
0.04466509446501732
0.003089234000071883
-0.03068244643509388
0.01929483190178871


### upload de documentos no comos-db

In [ ]:
import os
import json
from azure.cosmos import CosmosClient, exceptions
from dotenv import load_dotenv

# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

# Obter as variáveis de ambiente
URL = os.getenv('nosql-cosmos-uri')
KEY = os.getenv('nosql-cosmos-pk')

# Inicializar o cliente do Cosmos DB
cliente = CosmosClient(URL, credential=KEY)
database = cliente.get_database_client(DATABASE_NAME)
container = database.get_container_client(CONTAINER_NAME)


def process_and_upload_json_files(input_dir, container):
    for folder_name in os.listdir(input_dir):
        folder_path = os.path.join(input_dir, folder_name)
        if os.path.isdir(folder_path):
            for file_name in os.listdir(folder_path):
                if file_name.endswith('.json'):
                    file_path = os.path.join(folder_path, file_name)
                    
                    with open(file_path, 'r', encoding='utf-8') as json_file:
                        data = json.load(json_file)
                    
                    if 'id' not in data or not data['id']:
                        data['id'] = data['documento']  # Ajuste o campo id conforme necessário
                    
                    try:
                        container.read_item(item=data['id'], partition_key=data['id'])
                        print(f"Documento {data['id']} já existe. Pulando...")
                    except exceptions.CosmosResourceNotFoundError:
                        if not data.get('conteudoVector'):
                            # Aplicar a função de embeddings se o campo 'conteudoVector' não existir
                            texto = data.get('texto', '')
                            embedding, model = get_embeddings(texto)
                            data['conteudoVector'] = embedding
                            data['embeddingModel'] = model

                        # Salvar o arquivo JSON atualizado
                        with open(file_path, 'w', encoding='utf-8') as json_file:
                            json.dump(data, json_file, ensure_ascii=False, indent=4)

                        # Fazer o upload do JSON atualizado para o Azure Cosmos DB
                        try:
                            container.create_item(body=data)
                            print(f"Documento {data['id']} enviado com sucesso.")
                        except exceptions.CosmosHttpResponseError as e:
                            print(f"Erro ao criar documento {data['id']}: {str(e)}")


input_directory = 'caminho/dos/seus/documentos'

process_and_upload_json_files(input_directory, container)

# exemplo de saida
# Documento x.json já existe. Pulando...
# Documento y.json enviado com sucesso.


### lista total de documentos (total de paginas de documentos, uma vez que cada .json é apenas uma pagina de um pdf originalmente)

In [52]:
def list_documents(container):
        query = "SELECT c.documento FROM c "
        # query = "SELECT COUNT(DISTINCT c.documento) as distinct_count FROM c"
        items = list(container.query_items(
            query=query,
            enable_cross_partition_query=True
        ))
        return items
documentos = list_documents(container)
print(f"total de documentos: {len(documentos)}")
# for documento in documentos:
#         print(documento)

total de documentos: 656


In [53]:
def list_documents(container):
    query = '''SELECT VALUE COUNT(1)
               FROM (SELECT DISTINCT c.documento
                     FROM c)'''
    items = list(container.query_items(
        query=query,
        enable_cross_partition_query=True
    ))
    return items[0]  # The query returns a single count value.

distinct_count = list_documents(container)
print(f"Total de documentos (paginas): {distinct_count}")


Total de documentos (paginas): 656


# lista total de documentos INTEIROS diferentes 

In [54]:
def list_documents(container):
    query = '''SELECT VALUE COUNT(1)
               FROM (
                   SELECT DISTINCT LEFT(c.documento, INDEX_OF(c.documento, '_pagina'))
                   FROM c
               )'''
    
    items = list(container.query_items(
        query=query,
        enable_cross_partition_query=True
    ))
    return items

documentos = list_documents(container)
print(f"Total de documentos(inteiros e distintos):\n{documentos[0]}")


Total de documentos(inteiros e distintos):
66


### Exclui documentos

In [23]:
from azure.cosmos import exceptions

# ID do documento que deseja excluir
document_id = input("Digite o ID do documento que deseja EXCLUIR")

# Valor da chave de partição (deve ser igual ao valor do campo 'documento')
partition_key_value = document_id

try:
    # Exclusão do documento
    response = container.delete_item(
        item=document_id,
        partition_key=partition_key_value  # Passar diretamente o valor da chave de partição
    )
    print(f'Excluído com sucesso Documento com ID {document_id}.')
except exceptions.CosmosResourceNotFoundError:
    print(f'NÃO ENCONTRADO Documento com ID "{document_id}".')
except Exception as e:
    print(f'Erro ao excluir documento: {str(e)}')


NÃO ENCONTRADO Documento com ID "seila".


# lista data sources

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexerClient
from azure.search.documents.indexes.models import SearchIndexerDataContainer, SearchIndexerDataSourceConnection
key = ''
service_endpoint = ''
client = SearchIndexerClient(service_endpoint, AzureKeyCredential(key))
def list_data_source_connections():
    result = client.get_data_source_connections()
    names = [ds.name for ds in result]
    print("Found {} Data Source Connections in the service: {}".format(len(result), ", ".join(names)))
list_data_source_connections()
